### Overview
This notebook walks your through the process of creating clips with LLM prompts using spoken content of the video. 

Pick a video, decide your prompt, generate a new clip ⚡️

It's as simple as it sounds.

If you want to go extra mile you can score and rank your results, add Image Overlays or Audio overlays on these clips.

In [1]:
# But first, let's install the dependecies.
!pip install -r requirements.txt

### Choose the Video

You can either use a fresh video from Youtube etc. or choose an exisitng one already uploaded on your VideoDB collection.

In [2]:
import os
from dotenv import load_dotenv
from video_prompter import get_connection

# TODO: setup .env file
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

# connect to VideoDB
conn = get_connection()

### ----- Upload a fresh video --------- #####
def fresh_video(url):
    video = conn.upload(url=url)
    #index spoken content in the video
    video.index_spoken_words()
    return video

#### ------ run the prompt on video --------####
def videodb_prompter(video_id, prompt):
    video = get_video(video_id)
    #get all the segment of videos that are
    return video_prompter(video, prompt)

In [3]:
from videodb import play_stream

# ----Existing video case ----

# TODO: replace with your video id
# video_id = "m-replace-with-your-video-id-24-7"
video_id = 'm-ea0f53f3-a767-4058-8fb2-99df21d54917'


# ---- Fresh video case ----
# url = "https://www.youtube.com/watch?v=HpUR7-Oe1ss"
# video = fresh_video(url)
# video_id = video.id


#watch the original video
video = conn.get_collection().get_video(video_id)
video.player_url

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/1c9f0bc8-7258-4feb-a966-4323d879ccab.m3u8'

### Run your Prompt

To create a clip using the `text_prompter` function from a video, it's crucial to craft a specific prompt that will help identify the most relevant segments for your use case. This prompt should highlight the themes, topics, or specific phrases you're interested in. The function then analyzes the video's spoken content to find segments that match your criteria. 

Before you can use `text_prompter`, make sure the video's spoken content is indexed with the `video.index_spoken_words()` function. This prepares the video for analysis by making its spoken content searchable.

The `text_prompter` will return sentences or segments from the video that match your prompt. Review these to ensure they align with your needs. You can then use these segments to create your clip, focusing on the content that's most relevant to your use case.

In [68]:
from video_prompter import text_prompter

#Choose a prompt to create create clip. 
user_prompt = "find sentences where a deal is discussed"
result = text_prompter(video.get_transcript_text(), user_prompt)

{'id': 'chatcmpl-8vLx9bi3Wrw0rLvEoneiDHazyO6Kn', 'object': 'chat.completion', 'created': 1708679775, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "sentences": [\n    "Here\'s where I\'m at. $500,000 for 20% of a startup that has no revenue is way too high. You asked for too much. You made a strategic error. I\'m out.",\n    "And I\'m Lori Crowley, Cassidy\'s mom. And we\'re from Honolulu, Hawaii. I\'m seeking $50,000 in exchange for 50% of my. Company.",\n    "It\'s pretty rare that somebody comes in here and offers us 50%. Why did you do that? I mean, let\'s face it. I\'m only ten years old. I need a partner. I mean, I still have to go to high school.",\n    "Cassidy, I\'ve been doing little businesses since I was a kid. I can already tell from your presence, from your poise, from your grit that you\'re going to go very, very far."\n  ]\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 2752, 'completion_t

### Generate the Clip

To generate a clip, we'll  use **VideoDB**'s `keyword search` feature. We already leveraged the power of the LLM (Large Language Model) to identify relevant sentences. We'll use the search results to create a `programmable video stream`. Here's how you can approach this process:

We have the keywords in the `results` variable. Input these keywords into VideoDB's keyword search feature. This function will search through the indexed spoken content of your videos to find matches. 

The search will return a SearchResult object, which contains detailed information about the found segments, including their timestamps, the text of the spoken content, and possibly other metadata.

**Create a Programmable Video Stream with Timeline**: With the specific segments identified, you can now use the Timeline to create a new programmable video stream. The Timeline tool allows you to stitch together video segments based on the timestamps provided in the SearchResult object. You can arrange, cut, or combine these segments to craft a fresh video stream that focuses on your topic of interest.

In [44]:
# search matching video segments and watch the stream
from videodb import SearchType
from videodb.timeline import Timeline, VideoAsset, AudioAsset

timeline = Timeline(conn)
for clip_sentences in result:
    search_res = video.search(clip_sentences, search_type=SearchType.keyword)
    matched_segments = search_res.get_shots()
    
    #no exact match found.
    if len(matched_segments) == 0:
        continue

    #video segment
    video_shot = matched_segments[0]

    #Create a new Video Asset and add it to a timeline.
    timeline.add_inline(VideoAsset(asset_id=video.id, start=video_shot.start, end=video_shot.end))

In [42]:
stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://stream.videodb.io/v3/published/manifests/f9dc7b07-e5a7-4a9d-8ab9-33e86b78e12e.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/f9dc7b07-e5a7-4a9d-8ab9-33e86b78e12e.m3u8'

### Modify the Timeline

The programmable stream part of VideoDB allows you to not just watch the original clip but also modify and personalize the stream. Here we can add up the logo on each clip easily. You can read more about it here - https://docs.videodb.io/version-0-0-3-timeline-and-assets-44

In [49]:
# upload Image
from videodb import MediaType

image = conn.upload(url="https://www.freepnglogos.com/uploads/logo-ig-png/logo-ig-instagram-new-logo-vector-download-13.png", media_type=MediaType.image)
print(image)

Image(id=img-87fc6898-18ae-4d80-be51-cb12c7bd3dd1, collection_id=c-4adcd68a-50ef-4996-a37d-763af1d2d819, name=2hYi816d0St8fZ7RTNWeScRKpk)


In [50]:
from videodb.asset import VideoAsset, AudioAsset, ImageAsset

image_asset = ImageAsset(
    asset_id=image.id,
    width=100,
    height=100,
    x=80,
    y=20,
    duration=7
)

In [51]:
timeline.add_overlay(0, image_asset)
stream = timeline.generate_stream()
play_stream(stream)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/ef726137-6d9d-4497-b833-116a4d464861.m3u8'

### Bonus : Ranking using LLM
If you want to choose only a few top results and wodering how to do it, have LLM to rank your results and create a score that you can use to decide the order of segments. You can modify the ranking prompt creativiely to drive the outcome of it. We would love to see what you create 🙌🏼

In [60]:
from llm_agent import LLM
import re
import json
from math import floor

def ranking_prompt_llm(text, prompt):
    ranking_prompt = """Given the text provided below and a specific User Prompt, evaluate the relevance of the text
    in relation to the user's prompt. Please assign a relevance score ranging from 0 to 10, where 0 indicates no relevance 
    and 10 signifies perfect alignment with the user's request.
    The score quality also increases when the text is a complete senetence, making it perfect for a video clip result"""

    # pass the data
    ranking_prompt += f"""
    text: {text}
    User Prompt: {prompt}
    """

    # Add instructions to always return JSON at the end of processing.
    ranking_prompt += """
    Ensure the final output strictly adheres to the JSON format specified, without including additional text or explanations. 
    Use the following structure for your response:
    {
      "score": <relevance score>
    }
    """
    try:
        response = LLM().chat(message=ranking_prompt)
        print(response)
        output = response["choices"][0]["message"]["content"]
        res = json.loads(output)
        score = res.get('score')
        return score
    except Exception as e:
        return 0 

def rank_results(res, prompt, score_percentage=0.40):
    """
    rank and give score to each result
    """
    res_score = []
    for text in res:
        res_score.append((text, ranking_prompt_llm(text,prompt)))
    
    res_score_sorted = sorted(res_score, key=lambda x: x[1], reverse=True)
    return res_score_sorted[0: floor(len(res_score_sorted)*score_percentage)]

In [63]:
ranked_results = rank_results(result, user_prompt)

{'id': 'chatcmpl-8vLUCC1Eg5sAaEoM7HF2oGx7xvqXP', 'object': 'chat.completion', 'created': 1708677980, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "score": 9\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 195, 'completion_tokens': 9, 'total_tokens': 204}, 'system_fingerprint': None}
{'id': 'chatcmpl-8vLUEwptmGHRHDG4eKkxtBsgAL6OP', 'object': 'chat.completion', 'created': 1708677982, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "score": 10\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 174, 'completion_tokens': 9, 'total_tokens': 183}, 'system_fingerprint': None}
{'id': 'chatcmpl-8vLUGGYcDCmaXZt8e6RwK6cCE6Mwa', 'object': 'chat.completion', 'created': 1708677984, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"score": 0}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {

In [64]:
ranked_results

[("I'm seeking $50,000 in exchange for 50% of my. Company. Wow. Sharks.", 10),
 ('Sarah and Syed came in seeking $270,000 for 2% of their eco friendly cleaning product company, Blue Land. Lori and Daniel have offered 270,000 for 8%, and Kevin has offered 270,000 for 6%.',
  10),
 ("I'm going to make you an offer. But before I do, I want you to take a big breath because you're going to need it. Let's do it together. You're not going to like the number, and it's not going to be equity, that's for sure. I feel better about this offer than I even felt a few seconds ago. So here's what it is. You have good cash flow in this company, so you can support some debt. I'm going to give you the million bucks in a loan. 36 months, three years. Okay? 9% interest. And I want seven and a half percent equity.",
  10),
 ("You're allowed to counter? Yeah, or he'll pull it. Okay. What do you think? I think it's a very valid point. Right. So we'll do a million for 6% of the. Not 6% interest rate. 6%. The d

In [65]:
# search matching video segments and watch the stream
from videodb import SearchType
from videodb.timeline import Timeline, VideoAsset, AudioAsset

timeline = Timeline(conn)
for sentences, score in ranked_results:
    search_res = video.search(sentences, search_type=SearchType.keyword)
    matched_segments = search_res.get_shots()
    
    #no exact match found.
    if len(matched_segments) == 0:
        continue

    # get the first video matched segment
    video_shot = matched_segments[0]

    #Create a new Video Asset and add it to a timeline.
    timeline.add_inline(VideoAsset(asset_id=video.id, start=video_shot.start, end=video_shot.end))

In [66]:
stream = timeline.generate_stream()
play_stream(stream)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/8283a3a0-3df5-4f3b-b67b-805792139dd2.m3u8'

### Add some sound effects to it 🎶

Not just this we can jazz it up with audio overlays and create another stream with audio overlays.

In [67]:
# Add music overlay, this can be laughter soundtrack
audio = conn.upload(url="https://www.youtube.com/watch?v=q3VVxbJa61Q", media_type=MediaType.audio)

In [22]:
# 5 sec background music
background = AudioAsset(asset_id=audio.id, start=3, end=4, disable_other_tracks=True)

In [ ]:
timeline = Timeline(conn)
dur_so_far = 0
for clip_sentences in ranked_results:
    try:
        search_res = video.search(clip_sentences, search_type=SearchType.keyword)
        matched_segments = search_res.get_shots()
        
        #no exact match found.
        if len(matched_segments) == 0:
            continue
    
        #video segment
        video_shot = matched_segments[0]
    
        #Create a new Video Asset and add it to a timeline.
        timeline.add_inline(VideoAsset(asset_id=video.id, start=video_shot.start, end=video_shot.end))
    
        dur_so_far += (video_shot.end - video_shot.start)
        timeline.add_overlay(dur_so_far-2,background)
     except Exception as e:
        print(f"Error: skipping the segment {str(e)}")

In [24]:
#add music overlay in the last 2 sec of each supercut.
stream = timeline.generate_stream()
play_stream(stream)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/9cd651a1-7d52-42a0-b27b-deb385165c47.m3u8'

If you have any questions or feedback. Feel free to reach out to us 🙌🏼

* [Discord](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fdiscord.gg%2Fpy9P639jGz)
* [GitHub](https://github.com/video-db)
* [Website](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fvideodb.io)
* [Email](ashu@videodb.io)